In [ ]:
import fastf1 as f1

In [ ]:
def assign_sector(time_seconds, sector1_end_time, sector2_end_time, sector3_end_time):
    if time_seconds <= sector1_end_time:
        return 'Sector 1'
    elif time_seconds <= sector2_end_time:
        return 'Sector 2'
    else:
        return 'Sector 3'
f1.Cache.enable_cache('cache')

year = 2024
race = 'Race'
telemetry_data = []
for event in range(1,22):
    session = f1.get_session(year, event, race)
    session.load()
    for driver in session.drivers:
        # Get all laps for the driver
        driver_laps = session.laps.pick_driver(driver)

        if not driver_laps.empty:
            # Get the fastest lap
            fastest_lap = driver_laps.pick_fastest()
            if fastest_lap.empty:
                continue
            if (driver_laps[(driver_laps['LapNumber']==fastest_lap['LapNumber'])]['Sector1Time'].dt.seconds).empty:
                print(event, driver)
                print(driver_laps[(driver_laps['LapNumber']==fastest_lap['LapNumber'])]['Sector1Time'].dt.seconds)
                continue
            sec1time_sec = (driver_laps[(driver_laps['LapNumber']==fastest_lap['LapNumber'])]['Sector1Time'].dt.seconds).values[0]
            sec2time_sec = (sec1time_sec + driver_laps[(driver_laps['LapNumber']==fastest_lap['LapNumber'])]['Sector2Time'].dt.seconds).values[0]
            sec3time_sec = (sec2time_sec + driver_laps[(driver_laps['LapNumber']==fastest_lap['LapNumber'])]['Sector3Time'].dt.seconds).values[0]
            # Get telemetry data for the fastest lap
            telemetry = fastest_lap.get_telemetry()
            telemetry['TimeSeconds'] = telemetry['Time'].dt.total_seconds().astype(int)
            telemetry['Sector'] = telemetry['TimeSeconds'].apply(lambda x: assign_sector(
                                                    x,
                                                    sec1time_sec,
                                                    sec2time_sec,
                                                    sec3time_sec,) )
            telemetry['Event'] = event
            telemetry['DriverNumber'] = driver
            telemetry['X'] = telemetry['X'].astype(int)
            telemetry['Y'] = telemetry['Y'].astype(int)
            cols = ['TimeSeconds', 'X', 'Y', 'RPM', 'Speed', 'nGear', 'Throttle', 'Brake', 'Sector', 'Event', 'DriverNumber']
            telemetry_data.append(telemetry[cols])

all_telemetry_df = pd.concat(telemetry_data)
all_telemetry_df.to_csv('all_telemetry.csv', index=False)
print(all_telemetry_df)

In [ ]:

f1.Cache.enable_cache('cache')

year = 2024
event = 1
race = 'Race'
for event in range(1,25):
    session = f1.get_session(year, event, race)
    session.load()

lap = session.laps.pick_fastest()
tel = lap.get_telemetry()

sector1_end_time = lap['Sector1Time'].total_seconds()
sector2_end_time = sector1_end_time + lap['Sector2Time'].total_seconds()
sector3_end_time = sector2_end_time + lap['Sector3Time'].total_seconds()

def assign_sector(time_seconds):
    if time_seconds <= sector1_end_time:
        return 'Sector 1'
    elif time_seconds <= sector2_end_time:
        return 'Sector 2'
    else:
        return 'Sector 3'

tel['TimeSeconds'] = tel['Time'].dt.total_seconds()
tel['Sector'] = tel['TimeSeconds'].apply(assign_sector)

# ['Date', 'SessionTime', 'DriverAhead', 'DistanceToDriverAhead', 'Time',
# 'RPM', 'Speed', 'nGear', 'Throttle', 'Brake', 'DRS', 'Source',
# 'Distance', 'RelativeDistance', 'Status', 'X', 'Y', 'Z']

scale_in = -150.0
subj = tuple(tel[['X','Y']].itertuples(index=False, name=None))
pco = pyclipper.PyclipperOffset()
pco.AddPath(subj, pyclipper.JT_ROUND, pyclipper.ET_CLOSEDPOLYGON)

solution = pco.Execute(scale_in)

df = pd.DataFrame(solution[0], columns=['X_prime', 'Y_prime'])

# for all points in df, compute min distance
df['Original_Row'] = [
    np.argmin(np.linalg.norm(row - tel[['X','Y']], axis=1)) 
    for row in df.itertuples(index=False, name=None)
]
df['Speed'] = [tel.iloc[n]['Speed'] for n in df['Original_Row']]

fig1 = px.line(tel, 
              x='X', 
              y='Y', 
              color='Sector', 
              line_group = 'Sector',
              title=f'{year} {event} {race} Track with Sectors')

fig2 = px.scatter(df, 
              x='X_prime', 
              y='Y_prime',
              color='Speed',
              color_continuous_scale='Plasma',
               render_mode='webgl'
              )

fig = go.Figure(data = fig1.data + fig2.data)

fig.update_layout(
    xaxis=dict(visible=False),
    yaxis=dict(visible=False),
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
    showlegend=False,
    margin=dict(l=0, r=0, t=0, b=0),
    coloraxis_showscale=False
)
fig.update_yaxes(scaleanchor='x', scaleratio=1)

app = Dash(__name__)

app.layout = html.Div(children = [
    html.H1(children='F1 Track with Segments'),
    dcc.Graph(id='track-graph', figure=fig)])

if __name__ == '__main__':
    app.run_server(mode='inline', width='100px' )

In [ ]:


f1.Cache.enable_cache('cache')

year = 2023
event = 'Bahrain'
race = 'FP1'
session = f1.get_session(year, event, race)
session.load()

lap = session.laps.pick_fastest()
tel = lap.get_telemetry()

sector1_end_time = lap['Sector1Time'].total_seconds()
sector2_end_time = sector1_end_time + lap['Sector2Time'].total_seconds()
sector3_end_time = sector2_end_time + lap['Sector3Time'].total_seconds()
print(tel.columns)
def assign_sector(time_seconds):
    if time_seconds <= sector1_end_time:
        return 'Sector 1'
    elif time_seconds <= sector2_end_time:
        return 'Sector 2'
    else:
        return 'Sector 3'

tel['TimeSeconds'] = tel['Time'].dt.total_seconds()
tel['Sector'] = tel['TimeSeconds'].apply(assign_sector)

# ['Date', 'SessionTime', 'DriverAhead', 'DistanceToDriverAhead', 'Time',
# 'RPM', 'Speed', 'nGear', 'Throttle', 'Brake', 'DRS', 'Source',
# 'Distance', 'RelativeDistance', 'Status', 'X', 'Y', 'Z']

scale_in = -150.0
subj = tuple(tel[['X','Y']].itertuples(index=False, name=None))
pco = pyclipper.PyclipperOffset()
pco.AddPath(subj, pyclipper.JT_ROUND, pyclipper.ET_CLOSEDPOLYGON)

solution = pco.Execute(scale_in)

df = pd.DataFrame(solution[0], columns=['X_prime', 'Y_prime'])
# for all points in df, compute min distance
df['Original_Row'] = [
    np.argmin(np.linalg.norm(row - tel[['X','Y']], axis=1)) 
    for row in df.itertuples(index=False, name=None)
]
df['Speed'] = [tel.iloc[n]['Speed'] for n in df['Original_Row']]

fig1 = px.line(tel, 
              x='X', 
              y='Y', 
              color='Sector', 
              line_group = 'Sector',
              title=f'{year} {event} {race} Track with Sectors')

fig2 = px.scatter(df, 
              x='X_prime', 
              y='Y_prime',
              color='Speed',
              color_continuous_scale='Plasma',
               render_mode='webgl'
              )

fig = go.Figure(data = fig1.data + fig2.data)

fig.update_layout(
    xaxis=dict(visible=False),
    yaxis=dict(visible=False),
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
    showlegend=False,
    margin=dict(l=0, r=0, t=0, b=0),
    coloraxis_showscale=False
)
fig.update_yaxes(scaleanchor='x', scaleratio=1)

app = Dash(__name__)

app.layout = html.Div(children = [
    html.H1(children='F1 Track with Segments'),
    dcc.Graph(id='track-graph', figure=fig)])

if __name__ == '__main__':
    app.run_server(mode='inline', width='100px' )

In [ ]:
fig['data'][2]['showlegend']=False
fig['data'][2]['showlegend']=False

In [ ]:


f1.Cache.enable_cache('cache')

year = 2023
event = 'Bahrain'
race = 'FP1'
session = f1.get_session(year, event, race)
session.load()

lap = session.laps.pick_fastest()
tel = lap.get_telemetry()

sector1_end_time = lap['Sector1Time'].total_seconds()
sector2_end_time = sector1_end_time + lap['Sector2Time'].total_seconds()
sector3_end_time = sector2_end_time + lap['Sector3Time'].total_seconds()

def assign_sector(time_seconds):
    if time_seconds <= sector1_end_time:
        return 'Sector 1'
    elif time_seconds <= sector2_end_time:
        return 'Sector 2'
    else:
        return 'Sector 3'

tel['TimeSeconds'] = tel['Time'].dt.total_seconds()
tel['Sector'] = tel['TimeSeconds'].apply(assign_sector)

fig = px.line(tel, 
              x='X', 
              y='Y', 
              color='Sector', 
              line_group = 'Sector',
              title=f'{year} {event} {race} Track with Sectors')
fig.update_layout(
    xaxis=dict(visible=False),
    yaxis=dict(visible=False),
    plot_bgcolor='white',
    paper_bgcolor='white',
    showlegend=False,
    margin=dict(l=0, r=0, t=0, b=0)
)
fig.update_yaxes(scaleanchor='x', scaleratio=1)

app = Dash(__name__)

app.layout = html.Div(children = [
    html.H1(children='F1 Track with Segments'),
    dcc.Graph(id='track-graph', figure=fig)])

if __name__ == '__main__':
    app.run_server(mode='inline', width='100px' )

In [ ]:


f1.Cache.enable_cache('cache')

year = 2023
event = 'Bahrain'
race = 'FP1'
session = f1.get_session(year, event, race)
session.load()
drivers = session.drivers


driver_options = list()
for driver in drivers:
    driver_info = session.get_driver(driver)
    driver_name = f"{driver_info['FullName']} ({driver_info['Abbreviation']})"
    driver_options.append({'label': driver_name, 'value': driver})

app = Dash(__name__)

app.layout = html.Div(children = [
    html.H1(children='F1 Track with Segments'),
    html.Div([
        html.Label('Select driver:'),
        dcc.Dropdown(
            id='driver-dropdown',
            options=driver_options,
            value=drivers[0],
            clearable=False
        ),
    ]),
    
    dcc.Graph(id='track-graph', figure=fig),
])

@app.callback(
    Output('track-graph', 'figure'),
    Input('driver-dropdown', 'value')
)
def update_graph(selected_driver):
    driver_laps = session.laps.pick_drivers([selected_driver])
    if driver_laps.empty:
        return px.line(title=f'No data available for driver {selected_driver}')


    lap = driver_laps.pick_fastest()
    tel = lap.get_telemetry()
    
    sector1_end_time = lap['Sector1Time'].total_seconds()
    sector2_end_time = sector1_end_time + lap['Sector2Time'].total_seconds()
    sector3_end_time = sector2_end_time + lap['Sector3Time'].total_seconds()
    
    def assign_sector(time_seconds):
        if time_seconds <= sector1_end_time:
            return 'Sector 1'
        elif time_seconds <= sector2_end_time:
            return 'Sector 2'
        else:
            return 'Sector 3'
    
    tel['TimeSeconds'] = tel['Time'].dt.total_seconds()
    tel['Sector'] = tel['TimeSeconds'].apply(assign_sector)
    
    fig = px.line(tel, 
                  x='X', 
                  y='Y', 
                  color='Sector', 
                  line_group = 'Sector',
                  title=f'{year} {event} {race} Track with Sectors')
    
    fig.update_layout(
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        showlegend=False,
        margin=dict(l=0, r=0, t=0, b=0)
    )
    
    fig.update_yaxes(scaleanchor='x', scaleratio=1)

    return fig
    

if __name__ == '__main__':
    app.run_server(mode='inline', width='100px' )

In [ ]:
driver_info
# drivers

In [ ]:

f1.Cache.enable_cache('cache')

year = 2024
event = 1
race = 'Race'
session = f1.get_session(year, event, race)
session.load()
drivers = session.drivers


driver_options = list()
for driver in drivers:
    driver_info = session.get_driver(driver)
    driver_name = f"{driver_info['FullName']}"
    driver_position = f"{driver_info['ClassifiedPosition']}"
    driver_label_list = [
        html.Div(
            children=[
                html.Img(src=driver_info['HeadshotUrl'], style={'padding': 5}),
                html.Div(driver_position, style={'font-size': 32, 'position': 'absolute', 'top': '1px', 'left':'2px', 'color':'white'}),
                html.Br(),
                html.Span(driver_name , style={'font-size': 15, 'padding': 5, 'color':'white'})
            ],
            style={'padding': 3, 'position': 'relative', 'text-align': 'center'}
        )
    ]
    driver_options.append({'label': driver_label_list, 'value': driver})

telemetry_options = [{'label': 'Speed', 'value':'Speed'}, {'label': 'nGear', 'value': 'nGear'}, {'label': 'RPM', 'value':'RPM'}, {'label':'Throttle','value':'Throttle'}, {'label': 'Brake', 'value': 'Brake'}]

app = Dash(__name__)

app.layout = html.Div(children = [
    html.H1(children='F1 Track with Segments'),
    html.Div([
        html.Label('Select driver:'),
        dcc.Checklist(
            id='driver-dropdown',
            options=driver_options,
            inline=True
        ),
    ]),
    html.Div([
        html.Label('Select telemetry:'),
        dcc.Dropdown(
            id='telemetry-dropdown',
            options=telemetry_options,
            value=telemetry_options[0],
            clearable=False
        ),
    ]),
    
    dcc.Graph(id='track-graph', figure=fig),
])

@app.callback(
    Output('driver-dropdown', 'options'),
    Input('driver-dropdown', 'value')
)
def update_multi_options(value):
    options = driver_options
    if not value:
        return options
    if len(value) >= 2:
        options = [
            {
                'label': option['label'],
                'value': option['value'],
                'disabled': option['value'] not in value,
                                }
            for option in options
        ]
    return options

@app.callback(
    Output('track-graph', 'figure'),
    Input('driver-dropdown', 'value'),
    Input('telemetry-dropdown', 'value')
)
def update_graph(selected_driver, selected_telemetry):
    if not selected_telemetry:
        selected_telemetry = 'Speed'
        
    if not selected_driver:
        selected_driver = drivers[:1]
        driver_laps = session.laps.pick_drivers(selected_driver)
        if driver_laps.empty:
            return px.line(title=f'No data available for driver {selected_driver}')
    
    
        lap = driver_laps.pick_fastest()
        tel = lap.get_telemetry()
        
        sector1_end_time = lap['Sector1Time'].total_seconds()
        sector2_end_time = sector1_end_time + lap['Sector2Time'].total_seconds()
        sector3_end_time = sector2_end_time + lap['Sector3Time'].total_seconds()
        
        def assign_sector(time_seconds):
            if time_seconds <= sector1_end_time:
                return 'Sector 1'
            elif time_seconds <= sector2_end_time:
                return 'Sector 2'
            else:
                return 'Sector 3'
        
        tel['TimeSeconds'] = tel['Time'].dt.total_seconds()
        tel['Sector'] = tel['TimeSeconds'].apply(assign_sector)
        
        fig = px.line(tel, 
                      x='X', 
                      y='Y', 
                      color='Sector', 
                      line_group = 'Sector',
                      title=f'{year} {event} Track with Sectors')
        
        fig.update_traces(line=dict(width=10))

    elif len(selected_driver) == 1:
        driver_laps = session.laps.pick_drivers(selected_driver)
        if driver_laps.empty:
            return px.line(title=f'No data available for driver {selected_driver}')
    
    
        lap = driver_laps.pick_fastest()
        tel = lap.get_telemetry()
        
        sector1_end_time = lap['Sector1Time'].total_seconds()
        sector2_end_time = sector1_end_time + lap['Sector2Time'].total_seconds()
        sector3_end_time = sector2_end_time + lap['Sector3Time'].total_seconds()
        
        def assign_sector(time_seconds):
            if time_seconds <= sector1_end_time:
                return 'Sector 1'
            elif time_seconds <= sector2_end_time:
                return 'Sector 2'
            else:
                return 'Sector 3'
        
        tel['TimeSeconds'] = tel['Time'].dt.total_seconds()
        tel['Sector'] = tel['TimeSeconds'].apply(assign_sector)
        scale_in = -250.0
        subj = tuple(tel[['X','Y']].itertuples(index=False, name=None))
        pco = pyclipper.PyclipperOffset()
        pco.AddPath(subj, pyclipper.JT_ROUND, pyclipper.ET_CLOSEDPOLYGON)
        
        solution = pco.Execute(scale_in)
        
        df = pd.DataFrame(solution[0], columns=['X_prime', 'Y_prime'])
        
        # for all points in df, compute min distance
        df['Original_Row'] = [
            np.argmin(np.linalg.norm(row - tel[['X','Y']], axis=1)) 
            for row in df.itertuples(index=False, name=None)
        ]
        df[selected_telemetry] = [tel.iloc[n][selected_telemetry] for n in df['Original_Row']]
        
        fig1 = px.line(tel, 
                      x='X', 
                      y='Y', 
                      color='Sector', 
                      line_group = 'Sector',
                      title=f'{year} {event} Track with Sectors')
        
        fig2 = px.scatter(df, 
                      x='X_prime', 
                      y='Y_prime',
                      color=selected_telemetry,
                      color_continuous_scale='Plasma',
                       render_mode='webgl'
                      )
        
        fig1.update_traces(line=dict(width=10))
        fig = go.Figure(data = fig1.data + fig2.data)
        
    elif len(selected_driver) == 2:
        driver_1_laps = session.laps.pick_drivers(selected_driver[:1])
        driver_2_laps = session.laps.pick_drivers(selected_driver[1:])
    
        lap_d1 = driver_1_laps.pick_fastest()
        lap_d2 = driver_2_laps.pick_fastest()
        tel_d1 = lap_d1.get_telemetry()
        tel_d2 = lap_d2.get_telemetry()
        
        sector1_end_time = lap_d1['Sector1Time'].total_seconds()
        sector2_end_time = sector1_end_time + lap_d1['Sector2Time'].total_seconds()
        sector3_end_time = sector2_end_time + lap_d1['Sector3Time'].total_seconds()
        
        def assign_sector(time_seconds):
            if time_seconds <= sector1_end_time:
                return 'Sector 1'
            elif time_seconds <= sector2_end_time:
                return 'Sector 2'
            else:
                return 'Sector 3'
        
        tel_d1['TimeSeconds'] = tel_d1['Time'].dt.total_seconds()
        tel_d2['TimeSeconds'] = tel_d2['Time'].dt.total_seconds()
        tel_d1['Sector'] = tel_d1['TimeSeconds'].apply(assign_sector)
        tel_d2['Sector'] = tel_d2['TimeSeconds'].apply(assign_sector)
        
        scale_in = -250.0
        subj = tuple(tel_d1[['X','Y']].itertuples(index=False, name=None))
        pco = pyclipper.PyclipperOffset()
        pco.AddPath(subj, pyclipper.JT_ROUND, pyclipper.ET_CLOSEDPOLYGON)
        
        solution = pco.Execute(scale_in)
        
        df_in = pd.DataFrame(solution[0], columns=['X_prime', 'Y_prime'])
        
        # for all points in df, compute min distance
        df_in['Original_Row'] = [
            np.argmin(np.linalg.norm(row - tel_d1[['X','Y']], axis=1)) 
            for row in df_in.itertuples(index=False, name=None)
        ]
        df_in[selected_telemetry] = [tel_d1.iloc[n][selected_telemetry] for n in df_in['Original_Row']]
        
        scale_out = 250.0
        subj = tuple(tel_d2[['X','Y']].itertuples(index=False, name=None))
        pco = pyclipper.PyclipperOffset()
        pco.AddPath(subj, pyclipper.JT_ROUND, pyclipper.ET_CLOSEDPOLYGON)
        
        solution = pco.Execute(scale_out)
        
        df_out = pd.DataFrame(solution[0], columns=['X_prime', 'Y_prime'])
        
        # for all points in df, compute min distance
        df_out['Original_Row'] = [
            np.argmin(np.linalg.norm(row - tel_d2[['X','Y']], axis=1)) 
            for row in df_out.itertuples(index=False, name=None)
        ]
        df_out[selected_telemetry] = [tel_d2.iloc[n][selected_telemetry] for n in df_out['Original_Row']]
        
        fig1 = px.line(tel_d1, 
                      x='X', 
                      y='Y', 
                      color='Sector', 
                      line_group = 'Sector',
                      title=f'{year} {event} Track with Sectors')
        
        fig2 = px.scatter(df_in, 
                      x='X_prime', 
                      y='Y_prime',
                      color=selected_telemetry,
                      color_continuous_scale='Plasma',
                       render_mode='webgl'
                      )
        
        fig3 = px.scatter(df_out, 
                      x='X_prime', 
                      y='Y_prime',
                      color=selected_telemetry,
                      color_continuous_scale='Plasma',
                       render_mode='webgl'
                      )
        
        fig1.update_traces(line=dict(width=10))
        fig = go.Figure(data = fig1.data + fig2.data + fig3.data)
    
    fig.update_layout(
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        showlegend=False,
        margin=dict(l=0, r=0, t=0, b=0)
    )
    
    fig.update_yaxes(scaleanchor='x', scaleratio=1)


    return fig
    

if __name__ == '__main__':
    app.run_server(mode='inline', width='100px' )